In [1]:
import streamlit as st
import pandas as pd
import joblib
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from sklearn import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col

import getpass
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
from datetime import timedelta


import folium

X_final_scaled=pd.read_csv('x_final_scaled.csv')
truck_location_df=pd.read_csv('truck_manager_merged_df.csv')

In [2]:
truck_location_df['Truck_ID'].unique().tolist()

[1, 2, 13, 14, 17, 21, 27, 28, 33, 34, 35, 43, 44, 46, 47]

In [3]:
truck_location_df

,Unnamed: 0,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,...,TRUCK_ID,REVENUE,priority,current_location,next_start_time,location_visited,predicted_earning,total_distance_traveled,Name,City
0,0,1,2021-08-23,11,22,4,10,40,3289.0,12,...,1.0,4.618650e+04,15,3384.0,NaN,"[2314, 3195, 3384, 3384]","[21662.208984375, 12465.9833984375, 32482.3613...",8.384929,Corporate Owned,San Mateo
1,1,2,2021-08-23,10,21,3,9,27,3289.0,12,...,2.0,8.208100e+04,13,1796.0,NaN,"[2269, 1796, 1796]","[38517.40234375, 32610.1953125, 50182.0859375]",4.727919,Corporate Owned,San Mateo
2,2,13,2021-08-23,9,18,4,9,36,2255.0,10,...,13.0,7.846500e+04,14,3632.0,NaN,"[2998, 2998, 3632, 3632]","[23408.6875, 15517.5302734375, 9179.876953125,...",9.319448,Jacob Nguyen,San Mateo
3,3,14,2021-08-23,9,20,4,11,44,3289.0,12,...,14.0,1.077090e+05,12,3488.0,NaN,"[2900, 2900, 3488, 3488]","[39449.875, 25573.009765625, 23928.201171875, ...",3.736695,Jacob Nguyen,San Mateo
4,4,17,2021-08-23,8,23,3,9,27,3304.0,16,...,17.0,1.057530e+06,3,1730.0,NaN,"[1233, 1730, 1730]","[248674.71875, 169996.671875, 151586.09375]",15.177074,Corporate Owned,Denver
5,5,21,2021-08-23,9,22,3,8,24,3304.0,14,...,21.0,4.143250e+05,11,14800.0,NaN,"[1550, 1321, 14800]","[100875.8828125, 81790.90625, 82035.03125]",12.775782,Corporate Owned,Denver
6,6,27,2021-08-23,10,22,2,9,18,3304.0,13,...,27.0,9.136590e+05,5,15501.0,NaN,"[1056, 15501]","[136497.921875, 196333.09375]",7.225401,David Miller,Denver
7,7,28,2021-08-23,10,22,3,10,30,3304.0,13,...,28.0,6.698890e+05,10,1356.0,NaN,"[15457, 1356, 1356]","[74791.4609375, 84143.0390625, 75035.8984375]",5.437979,David Miller,Denver
8,8,33,2021-08-23,10,21,2,10,20,14997.0,12,...,33.0,1.047404e+06,4,15465.0,NaN,"[15019, 15465]","[144473.765625, 210992.921875]",8.345571,Corporate Owned,Seattle
9,9,34,2021-08-23,8,20,4,9,36,15098.0,13,...,34.0,7.243824e+05,8,15464.0,NaN,"[5176, 5176, 1808, 15464]","[87284.4921875, 52510.99609375, 31896.60742187...",15.025554,Kevin Cisneros,Seattle


In [4]:
truck_location_df["location_visited"] = truck_location_df["location_visited"].apply(eval)
truck_location_df["predicted_earning"] = truck_location_df["predicted_earning"].apply(eval)

In [5]:
truck_df_exploded = truck_location_df.explode(["location_visited", "predicted_earning"], ignore_index=True)

In [6]:
truck_df_exploded["shift"] = truck_df_exploded.groupby("Truck_ID").cumcount() + 1

In [7]:
all_locations = truck_df_exploded["location_visited"].tolist()

In [8]:
all_locations

[2314,
 3195,
 3384,
 3384,
 2269,
 1796,
 1796,
 2998,
 2998,
 3632,
 3632,
 2900,
 2900,
 3488,
 3488,
 1233,
 1730,
 1730,
 1550,
 1321,
 14800,
 1056,
 15501,
 15457,
 1356,
 1356,
 15019,
 15465,
 5176,
 5176,
 1808,
 15464,
 15472,
 5176,
 5176,
 5165,
 1805,
 3439,
 10398,
 8405,
 15370,
 13099,
 1282,
 3769]

In [12]:
filtered_df = X_final_scaled[X_final_scaled['LOCATION_ID'].isin(all_locations)]

In [13]:
filtered_df

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,Profit,LAT,LONG,LOCATION_ID,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
3632,33,3,15,0,22,0,2633.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3633,33,3,16,0,22,0,2209.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3634,33,3,17,0,22,0,4462.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3635,33,3,18,0,22,0,5705.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3636,33,3,19,0,22,0,3292.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520809,9,10,10,4,13,0,82.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520810,9,10,11,4,13,0,637.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520811,9,10,12,4,13,0,1570.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520812,9,10,13,4,13,0,819.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1


In [15]:
filtered_df.drop_duplicates()

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,Profit,LAT,LONG,LOCATION_ID,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
3632,33,3,15,0,22,0,2633.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3633,33,3,16,0,22,0,2209.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3634,33,3,17,0,22,0,4462.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3635,33,3,18,0,22,0,5705.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
3636,33,3,19,0,22,0,3292.5,47.647447,-122.336579,8405,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520809,9,10,10,4,13,0,82.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520810,9,10,11,4,13,0,637.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520811,9,10,12,4,13,0,1570.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1
520812,9,10,13,4,13,0,819.0,37.549981,-122.316965,2269,...,0,0,1,0,0,0,0,0,0,1


In [17]:
filtered_df.to_csv('X_final_scaled_Nathan.csv', index=False)

In [9]:
X_final_scaled

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,Profit,LAT,LONG,LOCATION_ID,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
0,68,7,8,6,10,0,10027.0,40.768807,-73.985856,3007,...,0,0,0,1,0,1,0,0,0,0
1,68,7,9,6,10,0,17477.0,40.768807,-73.985856,3007,...,0,0,0,1,0,1,0,0,0,0
2,68,7,10,6,10,0,13992.0,40.768807,-73.985856,3007,...,0,0,0,1,0,1,0,0,0,0
3,68,7,11,6,10,0,17616.0,40.768807,-73.985856,3007,...,0,0,0,1,0,1,0,0,0,0
4,68,7,12,6,10,0,15415.0,40.768807,-73.985856,3007,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540196,17,4,9,3,8,0,5711.0,39.750495,-104.932312,5115,...,0,1,0,0,0,0,0,1,0,0
540197,17,4,10,3,8,0,3741.0,39.750495,-104.932312,5115,...,0,1,0,0,0,0,0,1,0,0
540198,17,4,11,3,8,0,2786.0,39.750495,-104.932312,5115,...,0,1,0,0,0,0,0,1,0,0
540199,17,4,12,3,8,0,4843.0,39.750495,-104.932312,5115,...,0,1,0,0,0,0,0,1,0,0


In [8]:
# Find the latitude and longitude for each location
location_lat_long = {}
for location in all_locations:
    location_info = X_final_scaled[X_final_scaled["LOCATION_ID"] == location]
    if not location_info.empty:
        lat = location_info["LAT"].values[0]
        long = location_info["LONG"].values[0]
        location_lat_long[location] = (lat, long)

In [9]:
location_lat_long

{5120: (39.744553758875, -105.0460297918044),
 15501: (39.7297636031683, -104.9570999083016),
 15457: (39.7546837785723, -105.00740997592388),
 1356: (39.7391296512315, -104.98875342588528),
 14984: (47.68107764993707, -122.27808082557672),
 14985: (47.6048147774926, -122.287139298393),
 14987: (47.60811230968602, -122.30804261743522),
 8405: (47.64744728634749, -122.33657858578592),
 15370: (42.37791614850742, -71.04294881518484),
 3331: (42.3571671310234, -71.05542929198614),
 1282: (42.35097880323918, -71.10981657979718),
 3769: (42.32299649986056, -71.04394544772875)}

In [10]:
def get_lat_long(location_list):
    lat_list = []
    long_list = []
    for loc in location_list:
        if loc in location_lat_long:
            lat, long = location_lat_long[loc]
            lat_list.append(lat)
            long_list.append(long)
        else:
            lat_list.append(None)
            long_list.append(None)
    return lat_list, long_list

lat, lon = get_lat_long(truck_df_exploded["location_visited"])

truck_df_exploded["Lat"] = lat
truck_df_exploded["Lon"] = lon

In [11]:
import folium

In [12]:
m = folium.Map(location=[39.744553758875, -105.0460297918044],
               zoom_start=13)

In [13]:
truck_df_exploded

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,priority,current_location,next_start_time,location_visited,predicted_earning,shift,Lat,Lon
0,27,2021-08-23,9,19,2,9,18,3304,11,"[6, 5]","[9, 15, 19]",1,15501,NaN,5120,159078.1,1,39.744554,-105.046030
1,27,2021-08-23,9,19,2,9,18,3304,11,"[6, 5]","[9, 15, 19]",1,15501,NaN,15501,138177.03,2,39.729764,-104.957100
2,28,2021-08-24,8,22,3,9,27,3304,15,"[5, 5, 5]","[8, 13, 18, 22]",2,1356,NaN,15457,104917.62,1,39.754684,-105.007410
3,28,2021-08-24,8,22,3,9,27,3304,15,"[5, 5, 5]","[8, 13, 18, 22]",2,1356,NaN,1356,71965.97,2,39.739130,-104.988753
4,28,2021-08-24,8,22,3,9,27,3304,15,"[5, 5, 5]","[8, 13, 18, 22]",2,1356,NaN,1356,99176.91,3,39.739130,-104.988753
5,43,2021-08-25,10,19,3,8,24,4121,10,"[4, 3, 3]","[10, 14, 17, 19]",3,14985,NaN,14984,110745.87,1,47.681078,-122.278081
6,43,2021-08-25,10,19,3,8,24,4121,10,"[4, 3, 3]","[10, 14, 17, 19]",3,14985,NaN,14985,64369.984,2,47.604815,-122.287139
7,43,2021-08-25,10,19,3,8,24,4121,10,"[4, 3, 3]","[10, 14, 17, 19]",3,14985,NaN,14985,124440.8,3,47.604815,-122.287139
8,44,2021-08-26,10,22,3,11,33,1917,13,"[5, 4, 4]","[10, 15, 19, 22]",4,8405,NaN,14987,77580.87,1,47.608112,-122.308043
9,44,2021-08-26,10,22,3,11,33,1917,13,"[5, 4, 4]","[10, 15, 19, 22]",4,8405,NaN,8405,126884.69,2,47.647447,-122.336579


In [51]:

place_lat = truck_df_exploded['Lat'].astype(float).tolist()

place_lng = truck_df_exploded['Lon'].astype(float).tolist()


In [52]:
points = []

# read a series of points from coordinates and assign them to points object
for i in range(len(place_lat)):
    points.append([place_lat[i], place_lng[i]])

# specify an icon of your desired shape or chosing in place for the coordinates points
for index,lat in enumerate(place_lat):
    folium.Marker([lat, 
                   place_lng[index]],
                  popup=('Bus Station{} \n '.format(index))
                  ,icon = folium.Icon(color='blue',icon_color='white',prefix='fa', icon='truck-fast')
                  ).add_to(m)

In [53]:
folium.PolyLine(points, color = 'blue', dash_array = '5', opacity = '0.85',
                tooltip = 'Truck Route').add_to(m)

In [54]:
m

In [57]:
def filter_truck_data(truck_id):
    return truck_df_exploded[truck_df_exploded['Truck_ID'] == truck_id]

In [58]:
selected_truck_id = 44

In [60]:
selected_truck_data = filter_truck_data(selected_truck_id)

# FOLIUM MAP
m = folium.Map(location=[selected_truck_data['Lat'].iloc[0], selected_truck_data['Lon'].iloc[0]],
                zoom_start=13)

place_lat = selected_truck_data['Lat'].astype(float).tolist()
place_lng = selected_truck_data['Lon'].astype(float).tolist()

points = []

# read a series of points from coordinates and assign them to points object
for i in range(len(place_lat)):
    points.append([place_lat[i], place_lng[i]])

# specify an icon of your desired shape or choosing in place for the coordinates points
for index, lat in enumerate(place_lat):
    folium.Marker([lat, place_lng[index]],
                    popup=('Truck Location{} \n '.format(index)),
                    icon=folium.Icon(color='blue', icon_color='white', prefix='fa', icon='truck')
                    ).add_to(m)

folium.PolyLine(points, color='blue', dash_array='5', opacity='0.85',
                tooltip='Truck Route').add_to(m)

# Display the map with the updated position and points for the selected truck
m

In [65]:
import openrouteservice as ors

#ors client
ors_client = ors.Client(key='5b3ce3597851110001cf6248d282bc5c5d534216a412fa4e36a497e3')

# Define a function to get the route between two points using ORS
def get_route(start_point, end_point):
        radius = 10000  # 10 kilometers
        profile = 'driving-car'
        try:
                # Get the route between the start and end points
                route = ors_client.directions(
                coordinates=[start_point, end_point],
                profile=profile,
                format='geojson',
                radiuses=[radius, radius]
                )
                return route

        except ors.exceptions.ApiError as e:
                print(e)
                return None

st.header('Routing Map')
st.subheader('Choose a truck')
truck_ids = [27,43,28,44,46,47]

def filter_truck_data(truck_id):
    return truck_df_exploded[truck_df_exploded['Truck_ID'] == truck_id]

# Get the selected truck IDs from the user using a multiselect box
selected_truck_ids = st.multiselect("Select Truck IDs", truck_ids)

# Check if truck IDs are selected
if selected_truck_ids:
    st.success(f"Your selected Truck IDs '{selected_truck_ids}' have been saved!")

    # Get the data for the first selected truck
    selected_truck_data = filter_truck_data(selected_truck_ids[0])

    # FOLIUM MAP
    m = folium.Map(location=[selected_truck_data['Lat'].iloc[0], selected_truck_data['Lon'].iloc[0]], zoom_start=13)

    # Iterate through selected truck IDs to display each truck route
    for selected_truck_id in selected_truck_ids:
        # Filter truck data based on the selected truck ID
        selected_truck_data = filter_truck_data(selected_truck_id)

        place_lat = selected_truck_data['Lat'].astype(float).tolist()
        place_lng = selected_truck_data['Lon'].astype(float).tolist()

        points = []

        # read a series of points from coordinates and assign them to points object
        for i in range(len(place_lat)):
            points.append([place_lat[i], place_lng[i]])

        # specify an icon of your desired shape or choosing in place for the coordinates points
        for index, lat in enumerate(place_lat):
            folium.Marker([lat, place_lng[index]],
                          popup=('Truck Location {} \n '.format(index)),
                          icon=folium.Icon(color='blue', icon_color='white', prefix='fa', icon='truck')
                          ).add_to(m)

        # Choose a different polyline color for each truck route
        colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow']
        color_index = truck_ids.index(selected_truck_id)
        polyline_color = colors[color_index % len(colors)]

        folium.PolyLine(points, color=polyline_color, dash_array='5', opacity='0.85',
                        tooltip=f'Truck Route {selected_truck_id}').add_to(m)

        
        for i in range(len(place_lat) - 1):
                start_point = [place_lat[i], place_lng[i]]
                end_point = [place_lat[i + 1], place_lng[i + 1]]

                # Check if the start point and end point are the same
                if start_point != end_point:
                        # Get the route between two consecutive points
                        route = get_route(start_point, end_point)

                        # Extract the coordinates of the route
                        route_coords = route['features'][0]['geometry']['coordinates']

                        # Add the route as a polyline to the map
                        folium.PolyLine(route_coords, color=polyline_color, dash_array='5', opacity='0.85',
                                        tooltip=f'Truck Route {selected_truck_id}').add_to(m)
                        
    # Display the map with the selected truck routes
    st_folium(m, width=725)

2023-08-01 01:49:19.505 
  command:

    streamlit run c:\Users\Nathan\anaconda3\envs\py38_env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
